In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from copy import deepcopy
from collections import Counter
import json
from tqdm import tqdm
import glob

In [4]:
file_list = glob.glob('/media3/CRP8/TDE/data/DR_photometry/galactic/*.json')
file_list.sort()
print(len(file_list))

40886


In [5]:
path_dr = glob.glob('/media3/CRP8/TDE/data/DR_photometry_full/galactic/*.csv')
len(path_dr)

40886

In [37]:
path_test = ('/media3/CRP8/TDE/data/DR_photometry_full/galactic/ZTF24aaavbup.csv')
pd.read_csv(path_test)

,oid,expid,hjd,mjd,mag,magerr,catflags,filtercode,ra,dec,...,ccdid,qid,limitmag,magzp,magzprms,clrcoeff,clrcounc,exptime,airmass,programid


In [38]:
with open('/media3/CRP8/TDE/data/DR_photometry/galactic/ZTF24aaavbup.json', "r") as f:
    test_old = json.load(f)
test_old

{'563315400020522': {'meta': {'nobs': 43,
   'ngoodobs': 36,
   'duration': 41.87922000000253,
   'filter': 'zi',
   'fieldid': 563,
   'rcid': 59,
   'coord': {'ra': 95.40995025634766, 'dec': 21.319618225097656},
   'h3': {'10': 622567238194528255}},
  'lc': [{'mjd': 59140.46473,
    'mag': 16.888416,
    'magerr': 0.0140887,
    'clrcoeff': 0.1889554},
   {'mjd': 59140.47009,
    'mag': 16.896212,
    'magerr': 0.014136765,
    'clrcoeff': 0.18305486},
   {'mjd': 59140.48548,
    'mag': 16.878319,
    'magerr': 0.014026942,
    'clrcoeff': 0.19212726},
   {'mjd': 59151.41353,
    'mag': 16.897028,
    'magerr': 0.014141819,
    'clrcoeff': 0.18137066},
   {'mjd': 59151.43958,
    'mag': 16.902657,
    'magerr': 0.0141767645,
    'clrcoeff': 0.18507533},
   {'mjd': 59153.31323,
    'mag': 16.883589,
    'magerr': 0.014059101,
    'clrcoeff': 0.18656443},
   {'mjd': 59165.25723,
    'mag': 16.889797,
    'magerr': 0.014097186,
    'clrcoeff': 0.19155179},
   {'mjd': 59165.27794,
    'm

In [33]:
test = pd.read_csv(path_dr[0])
np.median(test['ra'].values)

np.float64(295.3242336)

In [19]:
np.unique(test['oid'].values)

array([ 728107100005574,  728207100007400, 1727115200012343,
       1727215200016839])

In [23]:
with open(file_list[0], "r") as f:
    test_old = json.load(f)
#test_old

{'768113200002578': {'meta': {'nobs': 850,
   'ngoodobs': 563,
   'duration': 2403.7072000000044,
   'filter': 'zg',
   'fieldid': 768,
   'rcid': 49,
   'coord': {'ra': 316.3904724121094, 'dec': 51.05747604370117},
   'h3': {'10': 621954540696928255}},
  'lc': [{'mjd': 58205.47235,
    'mag': 17.92966,
    'magerr': 0.04292152,
    'clrcoeff': -0.061312504},
   {'mjd': 58206.44069,
    'mag': 18.055397,
    'magerr': 0.04474062,
    'clrcoeff': -0.07499947},
   {'mjd': 58206.51975,
    'mag': 17.953743,
    'magerr': 0.043255463,
    'clrcoeff': -0.07566564},
   {'mjd': 58207.46151,
    'mag': 18.170816,
    'magerr': 0.046585552,
    'clrcoeff': -0.0858842},
   {'mjd': 58208.47244,
    'mag': 18.675468,
    'magerr': 0.057094865,
    'clrcoeff': -0.050681647},
   {'mjd': 58208.51039,
    'mag': 18.534752,
    'magerr': 0.053708892,
    'clrcoeff': -0.019960199},
   {'mjd': 58210.51425,
    'mag': 17.952946,
    'magerr': 0.043244313,
    'clrcoeff': -0.038670123},
   {'mjd': 58211.48

In [1]:
all_lcs = []
count = 0
# Filter mapping
filter_map = {"zg": ("g", 1), "zr": ("r", 2), "zi": ("i", 3)}

for i in tqdm(range(len(path_dr))):
    data = pd.read_csv(path_dr[i])

    ztf_id = path_dr[i].split('/')[-1].split('.')[0]

    ra = data['ra'].values
    dec = data['dec'].values
    mag = data['mag'].values
    mag_err = data['magerr'].values
    mjd = data['mjd'].values
    filters = data['filtercode'].values
    catflags = data['catflags'].values
    obj_id = data['oid'].values

    
    # Check for empty arrays
    if len(ra) == 0 or len(mag) == 0:
        count = count+1
        print(f"⚠️ Empty array found for ztf_id {ztf_id} (ra: {len(ra)}, mag: {len(mag)})")

    # #print('Dataset New:',len(mag),len(mag_err),len(catflags),ztf_id,len(obj_id))
    
    # #print('Importing Old dataset-------')
    # path_json = f'/media3/CRP8/TDE/data/DR_photometry/galactic/{ztf_id}.json'    
    # with open(path_json, "r") as f:
    #     data_old = json.load(f)

    # keys_to_check = np.unique(data['oid'].values)    

    # csv_ids_str = set(map(str, keys_to_check))
    # json_keys_str = set(data_old.keys())
    
    # if csv_ids_str != json_keys_str:
    #     print("Mismatch found!")
    #     print("CSV only:", csv_ids_str - json_keys_str)
    #     print("JSON only:", json_keys_str - csv_ids_str)
    #     print(json_keys_str)
    #     print(csv_ids_str)
    

#     # Convert filtercodes to g/r/i
#     bands = np.array([filter_map[f][0] for f in filters])
    


    ra_mean = np.median(ra)
    dec_mean = np.median(dec)

    # Append as dictionary (arrays stored as lists for DataFrame)
    all_lcs.append({
        "ztf_id": ztf_id,
        "ra_mean_dr": ra_mean,
        "dec_mean_dr": dec_mean,
        "mjd": mjd.tolist(),
        "mag": mag.tolist(),
        "magerr": mag_err.tolist(),
        "band": bands.tolist(),
        "catflags": catflags.tolist()
    })

# # # Build dataframe
df_all = pd.DataFrame(all_lcs)


NameError: name 'tqdm' is not defined

In [7]:
count

32

In [8]:
len(df_all)

NameError: name 'df_all' is not defined

In [20]:
df_all.to_csv('galactic_DR_Alert_withcatflag.csv')

In [21]:
df_all

,ztf_id,ra_mean,dec_mean,mjd,mag,magerr,band,catflags
0,ZTF19aaxifvw,295.324234,40.326094,"[58204.5143519, 58207.5097685, 58207.5227662, ...","[16.7076283, 16.6761169, 16.7013016, 16.685249...","[0.0144392345, 0.0143157234, 0.0144141503, 0.0...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[32768, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,ZTF19aadosog,264.425380,-3.260064,"[58204.5047222, 58204.5313542, 58207.4700579, ...","[15.3615971, 15.3990307, 15.397109, 15.390646,...","[0.0269933715, 0.0268484559, 0.0268558413, 0.0...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[0, 0, 0, 0, 0, 0, 32768, 0, 0, 0, 0, 0, 0, 0,..."
2,ZTF18actrkpz,112.869984,0.854739,"[58204.1836111, 58204.1975926, 58207.2177431, ...","[13.987051, 14.0066805, 13.9918499, 14.0148687...","[0.0106569408, 0.0106596276, 0.0106575815, 0.0...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[0, 0, 0, 0, 32768, 0, 32768, 0, 0, 0, 0, 3276..."
3,ZTF19abrkryh,288.278997,-22.143266,"[58229.5008218, 58232.5003588, 58258.4809722, ...","[17.0531788, 17.0086212, 17.0192261, 17.020351...","[0.0203975122, 0.0200282242, 0.0201147124, 0.0...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[32768, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,ZTF22aapkjuq,48.384165,61.270517,"[58206.1445718, 58298.468125, 58303.4488426, 5...","[17.2257824, 17.1839542, 17.2256432, 17.185871...","[0.0266686473, 0.0263112336, 0.0266674254, 0.0...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[32768, 0, 32768, 32768, 32768, 32768, 0, 0, 0..."
...,...,...,...,...,...,...,...,...
40881,ZTF18abmmwds,322.235445,66.474263,"[58206.4420833, 58206.472037, 58209.4573148, 5...","[18.2208443, 18.0681763, 17.6635284, 17.627731...","[0.0321904682, 0.0299819037, 0.025351949, 0.02...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[32768, 32768, 0, 0, 32768, 0, 0, 32768, 32768..."
40882,ZTF19acabkwk,254.301940,-4.084825,"[58203.5008102, 58206.4605671, 58206.4886806, ...","[13.5890265, 13.6498413, 13.6958265, 13.575630...","[0.0183401555, 0.018281037, 0.0182370152, 0.01...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[0, 0, 0, 32768, 0, 0, 0, 0, 32768, 0, 0, 0, 0..."
40883,ZTF22ablxsvb,21.248236,70.617090,"[58269.4718981, 58273.4698958, 58279.4664699, ...","[15.4977226, 15.5062399, 15.4915037, 15.520465...","[0.0149599891, 0.0149686178, 0.0149537576, 0.0...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[0, 0, 0, 0, 0, 0, 0, 0, 32768, 0, 0, 32768, 0..."
40884,ZTF23aacgkuh,284.388522,31.103310,"[58204.4287731, 58204.4484954, 58204.4755093, ...","[13.8040924, 13.7858267, 13.7897882, 13.771007...","[0.0113108633, 0.0112995794, 0.0113020204, 0.0...","[g, g, g, g, g, g, g, g, g, g, g, g, g, g, g, ...","[0, 0, 0, 0, 0, 0, 0, 0, 32768, 0, 0, 0, 0, 0,..."


# Alerts:

In [ ]:
data_alert = pd.read_parquet('/media3/CRP8/TDE/data/xmatch/galactic/')
data_alert = data_alert.groupby('objectId', sort=False, as_index=False).agg(list)
data_alert['top_class'] = [np.unique(data_alert['finkclass'].values[i]) for i in range(data_alert.shape[0])]
data_alert['ztf_id']= data_alert['objectId']
data_alert = data_alert.rename(columns={"ra": "ra_alert", "dec": "dec_alert"})
data_alert['ra_alert_mean'] = 0.0
data_alert['dec_alert_mean'] = 0.0

for i in range(len(data_alert)):
    # convert string to list
    ra_list = data_alert.loc[i, 'ra_alert']
    dec_list = data_alert.loc[i, 'dec_alert']

    # take mean
    data_alert.loc[i, 'ra_alert_mean'] = np.median(ra_list)
    data_alert.loc[i, 'dec_alert_mean'] = np.median(dec_list)
print('Number of objects in the Alert Dataset:',len(data_alert))